# UFPR
## Data Science Big Data
### Linguagens de Programação
Trabalho 1 - Usar Dataset 2  
Vencimento: sábado, 27 abr. 2024, 23:00  
Aluno: Alceu Eilert Nascimento  

## Escopo do trabalho: importar, limpar e analisar um dataset

Informações sobre o rendimento de alunos de um curso em algumas disciplinas ao longo dos anos.

O primeiro arquivo contido no Dataset Notas, `historico-alg1_SIGA_ANONIMIZADO.csv`, refere-se ao aproveitamento 
de estudantes na disciplina ALGORITMOS 1 entre os anos de 2011 e 2022.

A primeira coluna ("matricula") é composta por números inteiros, onde cada número representa um indivíduo. 
Assim, repetições nessa coluna indicam que o estudante fez mais de uma vez a mesma matéria.

Atenção: R-nota indica REPROVAÇÃO POR NOTA e R-freq REPROVAÇÃO POR FALTA. 

Se houver outro "status" para representar reprovação, este dever ser trocado para o rótulo adequado (R-nota ou R-freq). 


Frequências < 75 causam reprovação por falta;

Médias abaixo de 50 causam reprovação por nota.


In [2]:
# SETUP
import csv
import re
import pandas as pd
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [3]:
#------------------------------------------------------------------------
# GET DATA
#------------------------------------------------------------------------

# Caminho do arquivo CSV
path =  "/home/aenascimento/dsbd/lingprog/data/dsbd_trab2.csv"

df = pd.read_csv(path, encoding='UTF-8')
df


,matricula,codigo,disciplina,periodo,ano,curriculo,ch,nota,frequencia,status,tipo,observacao,natureza,situacaoDiscente,nomeTuma,codigoCurriculoSie
0,0,CI055,ALGORITMOS E ESTRUTURAS DE DADOS I,1,2016,Sim,60,67,96,Aprovado,TURMA,C,Obrigatória,Conclusão formatura,NaN,21A
1,1,CI1055,ALGORITMOS E ESTRUTURAS DE DADOS 1,1,2019,Sim,60,0,73,R-freq,TURMA,NaN,Obrigatória,Registro ativo,A,21A
2,1,CI1055,ALGORITMOS E ESTRUTURAS DE DADOS 1,2,2019,Sim,60,11,76,R-nota,TURMA,NaN,Obrigatória,Registro ativo,B,21A
3,1,CI1055,ALGORITMOS E ESTRUTURAS DE DADOS 1,2,2020,Sim,60,0,0,Cancelado,TURMA,NaN,Obrigatória,Registro ativo,B,21A
4,1,CI1055,ALGORITMOS E ESTRUTURAS DE DADOS 1,1,2020,Sim,60,0,0,Cancelado,TURMA,NaN,Obrigatória,Registro ativo,ERE 2 - D,21A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,569,CI055,ALGORITMOS E ESTRUTURAS DE DADOS I,1,2015,Sim,60,0,0,Aprovado,EQUIVALENCIA,NaN,Obrigatória,Evasão,NaN,21A
905,570,CI1055,ALGORITMOS E ESTRUTURAS DE DADOS 1,1,2022,Sim,60,70,100,Aprovado,TURMA,NaN,Obrigatória,Registro ativo,BCC3,21A
906,571,CI055,ALGORITMOS E ESTRUTURAS DE DADOS I,1,2018,Não,60,98,100,Aprovado,TURMA,NaN,-,Registro ativo,A,21A
907,571,CI1055,ALGORITMOS E ESTRUTURAS DE DADOS 1,1,2019,Sim,60,0,0,Aprovado,EQUIVALENCIA,NaN,Obrigatória,Registro ativo,NaN,21A


In [4]:

#------------------------------------------------------------------------
# CLEAN DATA
#------------------------------------------------------------------------
"""
Remove irrelevant data.
Convert data type.
Remove duplicates.
Clear formatting.
Fix errors.
Handle missing values.
"""
print("\n")
print("------------------------------------------------------------------")
print("------------------------------------------------------------------")
print("A. Limpado o dataframe do arquivo .CSV (dsbd_trab2_clean0.csv):")       
print("------------------------------------------------------------------")
print("------------------------------------------------------------------")
print("\n")
print("A. I. Premissas da analise:")
print(f"- Variáveis escolhidas: matricula, período, ano, nota, frequencia, status, tipo.")
print(f"- Variáveis desconsideradas: código, disciplina, curriculo, ch, obs, natureza, situacaoDiscente, nomeTurma, codigoCuriculoSie.")

# Colunas escolhidas
colunas_escolhidas = ['matricula', 'periodo', 'ano', 'nota', 'frequencia', 'status', 'tipo']

# Selecionando apenas as colunas escolhidas
df_limpo = df[colunas_escolhidas]
df_limpo

# Verificando dados faltantes
print("Dados Faltantes por Coluna:")
print(df_limpo.isnull().sum())

# Convertendo tipos de dados se necessário
df_limpo['nota'] = pd.to_numeric(df_limpo['nota'], errors='coerce')
df_limpo['frequencia'] = pd.to_numeric(df_limpo['frequencia'], errors='coerce')

df_limpo

# Verificar os primeiros registros após limpeza
print("DataFrame Limpo:")
print(df_limpo.head())
print(f"Total de registros após limpeza: {df_limpo.shape[0]}")

# Aplicando o filtro para manter apenas os registros que não são de 'EQUIVALENCIA' ou 'APROVEITAMENTO'
df_limpo = df_limpo[~df_limpo['tipo'].isin(['EQUIVALENCIA', 'APROVEITAMENTO'])]

# Verificando o resultado
print("DataFrame após remoção de outliers:")
print(df_limpo.head())
print(f"Total de registros após limpeza: {df_limpo.shape[0]}")




import numpy as np  # Importando NumPy


def check_and_correct_status(df):
    # Define a nova coluna 'expected_status' baseada nas condições dadas
    conditions = [
        (df['frequencia'] < 75),
        (df['frequencia'] >= 75) & (df['nota'] < 50),
        (df['frequencia'] >= 75) & (df['nota'] >= 50)
    ]
    choices = ['R-freq', 'R-nota', 'Aprovado']
    df['expected_status'] = np.select(conditions, choices, default=df['status'])

    # Identificar onde o status atual não coincide com o esperado
    mask = (df['status'] != df['expected_status']) & (df['status'] == 'Matriculado')
    
    # Aplicar correções
    df.loc[mask, 'status'] = df.loc[mask, 'expected_status']

    # Verificação de discrepâncias
    discrepant_statuses = df[df['status'] != df['expected_status']]
    if not discrepant_statuses.empty:
        print("Status corrigidos com discrepâncias:")
        print(discrepant_statuses)
    else:
        print("Todos os status estão corretos.")

    # Remover a coluna 'expected_status' após todas as verificações necessárias
    df.drop(columns=['expected_status'], inplace=True)

# Chamar a função para verificar e corrigir o status
check_and_correct_status(df_limpo)  # Substitua 'df_limpo' pelo nome do seu DataFrame
df_limpo





------------------------------------------------------------------
------------------------------------------------------------------
A. Limpado o dataframe do arquivo .CSV (dsbd_trab2_clean0.csv):
------------------------------------------------------------------
------------------------------------------------------------------


A. I. Premissas da analise:
- Variáveis escolhidas: matricula, período, ano, nota, frequencia, status, tipo.
- Variáveis desconsideradas: código, disciplina, curriculo, ch, obs, natureza, situacaoDiscente, nomeTurma, codigoCuriculoSie.
Dados Faltantes por Coluna:
matricula     0
periodo       0
ano           0
nota          0
frequencia    0
status        0
tipo          0
dtype: int64
DataFrame Limpo:
   matricula periodo   ano  nota  frequencia     status   tipo
0          0       1  2016    67          96   Aprovado  TURMA
1          1       1  2019     0          73     R-freq  TURMA
2          1       2  2019    11          76     R-nota  TURMA
3     

/tmp/ipykernel_12460/1985472281.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpo['nota'] = pd.to_numeric(df_limpo['nota'], errors='coerce')
/tmp/ipykernel_12460/1985472281.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpo['frequencia'] = pd.to_numeric(df_limpo['frequencia'], errors='coerce')


,matricula,periodo,ano,nota,frequencia,status,tipo
0,0,1,2016,67,96,Aprovado,TURMA
1,1,1,2019,0,73,R-freq,TURMA
2,1,2,2019,11,76,R-nota,TURMA
3,1,2,2020,0,0,Cancelado,TURMA
4,1,1,2020,0,0,Cancelado,TURMA
...,...,...,...,...,...,...,...
902,568,1,2020,0,0,Cancelado,TURMA
903,568,1,2021,88,100,Aprovado,TURMA
905,570,1,2022,70,100,Aprovado,TURMA
906,571,1,2018,98,100,Aprovado,TURMA
